In [38]:
import geopandas as gpd
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import pandas as pd 
from osgeo import gdal
import rasterio
from rasterio.features import rasterize
from rasterio.mask import mask
import random
from shapely.geometry import Point
from shapely.wkt import loads
from matplotlib.table import Table
from IPython.display import display, Markdown
from geopy.distance import great_circle
from heapq import nsmallest
from geopy.distance import geodesic
from itertools import combinations
import seaborn as sns
import contextily as ctx

from pyincore import IncoreClient, DataService, Dataset, FragilityService, MappingSet
from pyincore.utils.dataprocessutil import DataProcessUtil
# importing pyIncone analyses:
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.combinedwindwavesurgebuildingdamage import CombinedWindWaveSurgeBuildingDamage

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [4]:
# Connect to IN-CORE Services
client = IncoreClient()

data_service = DataService(client)

Connection successful to IN-CORE services. pyIncore version detected: 1.13.0


# Data preparation

In [5]:
# Loading Galveston building inventory
bldg_dataset_id = "63ff6b135c35c0353d5ed3ac"
buildings = Dataset.from_data_service(bldg_dataset_id, data_service)
bld_invtry = buildings.get_dataframe_from_shapefile()
bld_invtry.head()
#bld_invtry.shape

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


,strctid,parid,struct_typ,year_built,no_stories,a_stories,b_stories,bsmt_type,sq_foot,gsq_foot,...,ffe_elev,g_elev,archetype,arch_wind,arch_flood,arch_sw,csv_guid,csv_sector,csv_val_st,geometry
0,STf3a21b5f-0334-4d77-911f-faa9a45c92d7,0,NaN,1980,2,0,0,0,47927,0,...,3.770,3.465,0,15,6,15,NaN,NaN,NaN,POINT (-94.81680 29.31409)
1,ST80ab0289-8c72-4699-b12b-08ef2f2a5c91,0,NaN,1954,1,0,0,1,100796,0,...,2.260,1.956,0,15,6,15,NaN,NaN,NaN,POINT (-94.82295 29.29635)
2,ST7916eaa8-0bfa-4b32-84ff-20f19a520f5c,0,NaN,1970,1,0,0,1,27121,0,...,2.680,2.375,0,15,6,15,7916eaa8-0bfa-4b32-84ff-20f19a520f5c,IRetail,2435054.5,POINT (-94.82423 29.29098)
3,ST79008971-a534-4e45-9e40-43a2a904e59b,0,NaN,2002,1,0,0,1,126333,0,...,2.915,2.610,0,15,6,15,79008971-a534-4e45-9e40-43a2a904e59b,IProfSer,4483599,POINT (-94.84045 29.29117)
4,ST2ad8ab6f-057f-468e-9911-6e92d9d4d01f,0,NaN,2002,1,0,0,1,114544,0,...,3.738,3.433,0,15,6,15,2ad8ab6f-057f-468e-9911-6e92d9d4d01f,IRealE,282702.06,POINT (-94.84242 29.29059)


In [11]:
# Removing any properties that have NaN values and values less than $1000 in column "appr_bldg"
# (the threshold is based on Galveston County's tax assessor data for mobile homes)
bld_invtry = bld_invtry[bld_invtry['appr_bldg'].notna() & (bld_invtry['appr_bldg'] >= 1000)]
#bld_invtry.shape

## Max damage state from Combined Building Damage Analyses

### TODO: Need to improve explanation and comments

In [31]:
def calculate_combined_building_damage(hazard_type, hazard_id, bldg_dataset_id, wind_mapping_id,
                                      surge_wave_mapping_id, flood_mapping_id):

    # Wind building damage
    fragility_service = FragilityService(client)
    wind_mapping_set = MappingSet(fragility_service.get_mapping(wind_mapping_id))
    w_bldg_dmg = BuildingDamage(client)
    w_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
    w_bldg_dmg.set_input_dataset('dfr3_mapping_set', wind_mapping_set)
    w_bldg_dmg.set_parameter("result_name", "Galveston-wind-dmg")
    w_bldg_dmg.set_parameter("hazard_type", hazard_type)
    w_bldg_dmg.set_parameter("hazard_id", hazard_id)
    w_bldg_dmg.set_parameter("num_cpu", 8)
    w_bldg_dmg.run_analysis()

    # surge-wave building damage
    surge_wave_mapping_set = MappingSet(fragility_service.get_mapping(surge_wave_mapping_id))
    sw_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
    sw_bldg_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)
    sw_bldg_dmg.set_parameter("result_name", "Galveston-sw-dmg")
    sw_bldg_dmg.set_parameter("hazard_type", hazard_type)
    sw_bldg_dmg.set_parameter("hazard_id", hazard_id)
    sw_bldg_dmg.set_parameter("num_cpu", 8)
    sw_bldg_dmg.run_analysis()

    # Flood mapping damage
    flood_mapping_set = MappingSet(fragility_service.get_mapping(flood_mapping_id))
    f_bldg_dmg = BuildingDamage(client)
    f_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
    f_bldg_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)
    f_bldg_dmg.set_parameter("result_name", "Galveston-flood-dmg")
    f_bldg_dmg.set_parameter("hazard_type", hazard_type)
    f_bldg_dmg.set_parameter("hazard_id", hazard_id)
    f_bldg_dmg.set_parameter("num_cpu", 8)
    f_bldg_dmg.run_analysis()

    # Running combined damage analysis
    surge_wave_damage = sw_bldg_dmg.get_output_dataset("ds_result")
    wind_damage = w_bldg_dmg.get_output_dataset("ds_result")
    flood_damage = f_bldg_dmg.get_output_dataset("ds_result")
    
    combined_bldg_dmg = CombinedWindWaveSurgeBuildingDamage(client)
    result_name = "Galveston-combined-dmg"
    combined_bldg_dmg.set_input_dataset("surge_wave_damage", surge_wave_damage)
    combined_bldg_dmg.set_input_dataset("wind_damage", wind_damage)
    combined_bldg_dmg.set_input_dataset("flood_damage", flood_damage)
    combined_bldg_dmg.set_parameter("result_name", result_name)
    combined_bldg_dmg.run_analysis()

    # Returning combined damage result 
    combined_dmg = combined_bldg_dmg.get_output_dataset("ds_result")
    combined_dmg_df = combined_dmg.get_dataframe_from_csv(low_memory=False)

    return combined_dmg_df

## Get max damage states

In [34]:
# Hurricane Ike Building combined damage

wind_mapping_id = "62fef3a6cef2881193f2261d"
surge_wave_mapping_id = "6303e51bd76c6d0e1f6be080"
flood_mapping_id = "62fefd688a30d30dac57bbd7"


hazard_type = "hurricane"
hazard_id = "5fa5a228b6429615aeea4410"
bldg_dataset_id = "63ff6b135c35c0353d5ed3ac"

combined_dmg_ike = calculate_combined_building_damage(hazard_type, hazard_id, bldg_dataset_id, wind_mapping_id,
                                      surge_wave_mapping_id, flood_mapping_id)

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


In [36]:
# Hurricane Ike 100 year Building combined damage
wind_mapping_id = "62fef3a6cef2881193f2261d"
surge_wave_mapping_id = "6303e51bd76c6d0e1f6be080"
flood_mapping_id = "62fefd688a30d30dac57bbd7"


hazard_type = "hurricane"
hazard_id = "5fa5a9497e5cdf51ebf1add2"
bldg_dataset_id = "63ff6b135c35c0353d5ed3ac"

combined_dmg_ike_100yrs = calculate_combined_building_damage(hazard_type, hazard_id, bldg_dataset_id, wind_mapping_id,
                                      surge_wave_mapping_id, flood_mapping_id)

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...
Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


In [39]:
# Hurricane Ike's building damage model output
ike = DataProcessUtil.get_max_damage_state(combined_dmg_ike)
# Hurricane Ike's 100-year event output building damage model 
event_100year = DataProcessUtil.get_max_damage_state(combined_dmg_ike_100yrs)

### Selection Criterion 1: Past flooding
#### - To identify buildings with past flooding, we'll use Hurricane Ike's hindcast model (hazard ID: 5fa5a228b6429615aeea4410)
#### - Of all buildings in the building inventory, we are only interested in those with DS_3 state that are residential.

In [40]:
# Merging Ike's damage states with building inventory using guid
ike_invtry = pd.merge(bld_invtry, ike, on='guid', how='outer')
#ike_invtry.shape

In [41]:
cluster = Dataset.from_data_service("63dc1f1362b9d001e6a1b485", data_service).get_dataframe_from_csv()
cluster.head()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


,arch_wind,category,cluster
0,12.000,Critical Facilities,Critical Medical - Acute Care Hospitals
1,14.000,Critical Facilities,Emergency Operations Centers
2,11.000,Critical Facilities,Critical Government - First Responder Facilities
3,NaN,Critical Facilities,"Non-ambulatory Facilities - Prisons, nursing homes, etc."
4,NaN,Emergency Housing,Emergency Shelters


In [42]:
# Keeping only residential buildings (identified from the arch_wind column)
ike_invtry = ike_invtry[ike_invtry['arch_wind'].isin([1, 2, 3, 4, 5, 17])]
#ike_invtry.shape

In [43]:
ike_invtry = ike_invtry.rename(columns={'max_state': 'max_state_ike'})

In [44]:
# Removing properties with damage states 0, 1, 2 and Keeping buildings with complete damage (DS_3)
ike_invtry_ds3 = ike_invtry[ike_invtry['max_state_ike'] == 'DS_3']
#ike_invtry_ds3.shape

### Selection Criterion 2: Predicted (future) flooding
#### - To identify buildings with future/predicted flooding, we'll use the 100-year simulation model (hazard ID: 5fa5a9497e5cdf51ebf1add2) and merge it with 
#### - Of all buildings in the building inventory, we are only interested in those with DS_3 state that are residential.

In [46]:
# Merging 100-year event output with Ike's outout
ike_100year = pd.merge(event_100year, ike_invtry_ds3, on='guid', how='inner')
#ike_100year_invtry.shape

In [47]:
ike_100year = ike_100year.rename(columns={'max_state': 'max_state_100year'})

In [48]:
# Only keeping buildings with projected complete damages (DS_3) due to the 100-year event
ike_100year_ds3 = ike_100year[ike_100year['max_state_100year'] == 'DS_3']
#ike_100year_ds3.shape

### Selection Criterion 3: Reality check
#### - Making sure the appraisal value of selected properties from previous step falls within FEMA's previously funded buyout projects 
#### - Removing any properties with "appr_bldg" value more than 321291.600 (this value is based on FEMA's HMA properties dataset with 50 - 99% damages cap for Texas after removing outliers -  and shows the maximum purchase offer to residential buildings owners from 2000 to 2020)

In [49]:
ike_100year_ds3_below_fema_cap = ike_100year_ds3[ike_100year_ds3['appr_bldg'] <= 321291.600]
ike_100year_ds3_below_fema_cap = ike_100year_ds3_below_fema_cap[["guid","appr_bldg","max_state_100year","max_state_ike","geometry"]]

In [50]:
ike_100year_ds3_below_fema_cap.head()

,guid,appr_bldg,max_state_100year,max_state_ike,geometry
0,e89d271f-e9b0-4599-ad65-99f8f285e2fc,290636.000,DS_3,DS_3,POINT (-94.84187 29.27899)
1,97c0a381-4894-420d-9933-ad4e34098170,289220.000,DS_3,DS_3,POINT (-94.84406 29.26691)
3,590c1b06-a3d6-4e6a-9480-d44a3a4d06d2,269628.000,DS_3,DS_3,POINT (-94.77322 29.32267)
4,8c6fd727-f3ce-4a8b-85da-2e2d01646ca1,222426.000,DS_3,DS_3,POINT (-94.77376 29.32268)
5,b23ded68-acb6-48a1-8fb5-c5902ee93994,254491.000,DS_3,DS_3,POINT (-94.77328 29.32246)


In [51]:
hua = Dataset.from_data_service("63ff8e895367c2261b4cb2ef", data_service).get_dataframe_from_csv()
hua.head()

Dataset already exists locally. Reading from local cached zip.
Unzipped folder found in the local cache. Reading from it...


,huid,blockid,bgid,tractid,FIPScounty,numprec,ownershp,race,hispan,family,...,hhinc,randincome,poverty,BLOCKID10_str,guid,placeNAME10,huestimate,x,y,geometry
0,B481677240001010H001,481677240001010,481677240001,48167724000,48167,1,2.000,1.000,0.000,0.000,...,1,6567.000,1.000,B481677240001010,8f1a6a6b-9258-46da-85a9-7a821f3d3c07,Galveston,71,-94.818,29.317,POINT (-94.8182653346525 29.3168730983553)
1,B481677240001010H002,481677240001010,481677240001,48167724000,48167,1,2.000,1.000,0.000,0.000,...,2,18898.000,0.000,B481677240001010,8f1a6a6b-9258-46da-85a9-7a821f3d3c07,Galveston,71,-94.818,29.317,POINT (-94.8182653346525 29.3168730983553)
2,B481677240001010H003,481677240001010,481677240001,48167724000,48167,1,2.000,1.000,0.000,0.000,...,1,13988.000,0.000,B481677240001010,8f1a6a6b-9258-46da-85a9-7a821f3d3c07,Galveston,71,-94.818,29.317,POINT (-94.8182653346525 29.3168730983553)
3,B481677240001010H004,481677240001010,481677240001,48167724000,48167,1,2.000,1.000,0.000,0.000,...,1,3570.000,1.000,B481677240001010,8f1a6a6b-9258-46da-85a9-7a821f3d3c07,Galveston,71,-94.818,29.317,POINT (-94.8182653346525 29.3168730983553)
4,B481677240001010H005,481677240001010,481677240001,48167724000,48167,1,2.000,1.000,0.000,0.000,...,2,16463.000,0.000,B481677240001010,0fdd9a62-74f5-4e4d-8bb6-9fcfb7142fae,Galveston,0,-94.818,29.317,POINT (-94.8182840451795 29.3166001186135)


In [52]:
# Merging housing unit allocation with selected properties guids
hua_ike_100year_ds3_below_fema_cap = pd.merge(ike_100year_ds3_below_fema_cap, hua, on='guid', how='left')
#hua_ike_100year_ds3_below_fema_cap.shape

In [53]:
# Excluding observations with "numprec" 0 or blank
hua_ike_100year_ds3_below_fema_cap = hua_ike_100year_ds3_below_fema_cap.loc[(hua_ike_100year_ds3_below_fema_cap['numprec'] != 0) & (~hua_ike_100year_ds3_below_fema_cap['numprec'].isna())]
#hua_ike_100year_ds3_below_fema_cap.shape

In [54]:
# Removing any rows with NAN values in column "Race"
hua_ike_100year_ds3_below_fema_cap = hua_ike_100year_ds3_below_fema_cap.dropna(subset=['race'])

In [55]:
# Renaming the columns
hua_ike_100year_ds3_below_fema_cap = hua_ike_100year_ds3_below_fema_cap.rename(columns={'geometry_x': 'geometry_bldg_invtry', 'geometry_y': 'geometry_hua'})

In [56]:
# Population dislocation
dislocation = pd.read_csv("popDislocation_multihazard.csv")

In [57]:
# Merging with population dislocation model
df_final = pd.merge(hua_ike_100year_ds3_below_fema_cap, dislocation[['huid', 'dislocated_combined_dmg']], on='huid', how='left')
#df_final.shape

In [58]:
df_final.head()

,guid,appr_bldg,max_state_100year,max_state_ike,geometry_bldg_invtry,huid,blockid,bgid,tractid,FIPScounty,...,hhinc,randincome,poverty,BLOCKID10_str,placeNAME10,huestimate,x,y,geometry_hua,dislocated_combined_dmg
0,97c0a381-4894-420d-9933-ad4e34098170,289220.000,DS_3,DS_3,POINT (-94.84406 29.26691),B481677257001017H028,481677257001017.000,481677257001.000,48167725700.000,48167.000,...,5.000,120294.000,0.000,B481677257001017,Galveston,1.000,-94.844,29.267,POINT (-94.8440602900364 29.2669113929446),True
1,590c1b06-a3d6-4e6a-9480-d44a3a4d06d2,269628.000,DS_3,DS_3,POINT (-94.77322 29.32267),B481677242001004H011,481677242001004.000,481677242001.000,48167724200.000,48167.000,...,5.000,100235.000,0.000,B481677242001004,Galveston,1.000,-94.773,29.323,POINT (-94.7732187400678 29.3226677367542),False
2,8c6fd727-f3ce-4a8b-85da-2e2d01646ca1,222426.000,DS_3,DS_3,POINT (-94.77376 29.32268),B481677242001001H017,481677242001001.000,481677242001.000,48167724200.000,48167.000,...,4.000,99349.000,0.000,B481677242001001,Galveston,5.000,-94.774,29.323,POINT (-94.7737557351818 29.3226770378826),True
3,8c6fd727-f3ce-4a8b-85da-2e2d01646ca1,222426.000,DS_3,DS_3,POINT (-94.77376 29.32268),B481677242001001H156,481677242001001.000,481677242001.000,48167724200.000,48167.000,...,3.000,61826.000,0.000,B481677242001001,Galveston,5.000,-94.774,29.323,POINT (-94.7737557351818 29.3226770378826),False
4,8c6fd727-f3ce-4a8b-85da-2e2d01646ca1,222426.000,DS_3,DS_3,POINT (-94.77376 29.32268),B481677242001001H157,481677242001001.000,481677242001.000,48167724200.000,48167.000,...,3.000,50266.000,0.000,B481677242001001,Galveston,5.000,-94.774,29.323,POINT (-94.7737557351818 29.3226770378826),True


In [59]:
# Converting the dataset to GeoDataFrame using the goemetry column 
df_final['geometry'] = df_final['geometry_bldg_invtry']
df_final = gpd.GeoDataFrame(df_final, geometry='geometry')

In [60]:
# We want to create a new column showing the appraisal value of each building ('appr_bldg' divided by the number of times a guid is repeated) 
# For the instances that a structure has more than one housing units.
df_final['count'] = df_final.groupby('guid')['guid'].transform('count')
df_final['housing_unit_appraisal_value'] = df_final['appr_bldg'] / df_final['count']

In [61]:
#df_final.columns

In [62]:
# Dropping the unnecassary columns
df_final = df_final.drop(['blockid','bgid','tractid','FIPScounty',
                          'gqtype','BLOCKID10_str','placeNAME10','geometry_hua'], axis=1)

In [63]:
df_final.rename(columns={'appr_bldg':'building_appraisal_value','ownershp':'ownership',
                         'dislocated_combined_dmg':'dislocated','count':'number_of_housing_units'}, inplace=True)

In [64]:
df_final = df_final[['guid', 'huid', 'building_appraisal_value', 'housing_unit_appraisal_value', 'geometry', 'number_of_housing_units',
                     'numprec', 'ownership', 'race', 'hispan', 'family', 'vacancy', 'incomegroup', 'hhinc', 'randincome',
                     'poverty', 'huestimate', 'dislocated' ,'max_state_100year', 'max_state_ike', 'x', 'y',]]

## Results

### 1) Tenure status

In [65]:
df_final['ownership'] = df_final['ownership'].replace({1: 'Owner', 2: 'Renter'})

owners = (df_final['ownership'] == 'Owner').sum()
renters = (df_final['ownership'] == 'Renter').sum()

normal_offer_owners = round(df_final.loc[df_final['ownership'] == 'Owner', 'housing_unit_appraisal_value'].sum())
normal_offer_renters = round(df_final.loc[df_final['ownership'] == 'Renter', 'housing_unit_appraisal_value'].sum())

benefit_owner = df_final.loc[(df_final['dislocated'] == True) & (df_final['ownership'] == 'Owner'), 'dislocated'].sum()
benefit_renter = df_final.loc[(df_final['dislocated'] == True) & (df_final['ownership'] == 'Renter'), 'dislocated'].sum()


ownership_summary = {
    '': ['# candidate housing units', 'Total purchase offer', 'Potential benefits (# people not dislocated by a simulated 100-year event)',
         'Potential challenges', 'Potential consequences', 'Equity considerations'],
    'Owners': [owners, f"${normal_offer_owners:,.2f}", benefit_owner,
                '(1) Program participation depends on financial standing, place-based attachments, risk perception, flood exposure, family composition, and community ties. <br/>'+
                '(2) Homeowners are less likely to participate if they have a mortgage. <br/>'+
                '(3) Elderly homeowners (with paid off mortgages) may oppose relocation, due to unwillingness to take out a second mortgage for a more expensive home and place-based attachments. <br/>'+
                '(4) Homeowners tend to decline offers if their property has undergone improvements, as home improvements are not factored into the market value. <br/>'+
                '(5) Lower chance of accepting an offer if cost sharing is required. <br/>'+
                '(6) Lower chance of accepting an offer if their house has undergone repair and reconstruction.'
                '(7) Owners of substantially damaged properties may have limited post-disaster rebuilding options. While eminent domain is prohibited in buyout programs, the owners  are legally restricted from rebuilding unless they flood-proof the structure or relocate, increasing their participation likelihood. <br/>'+
                '(8) Homeowners are less inclined to participate if the purchase offer is lower than their expected property value. <br/>'+
                '(9) Homeowners with inadequate property documentation are less likely to participate.', 
                '(1) Possibility of unaffordable housing options upon relocation <br/>'+
                '(2) Losing social network if relocated outside their community', 
                '(1) Sufficient financial incentives can alleviate post-buyout financial burdens and encourage program participation. <br/>'+
                '(2) Up to $22,500 compensation under URA for additional payments such as comparable homes, closing costs, or increased interest costs. <br/>'+
                '(3) Basement coverage through FEMA’s NFIP is limited, and compensation may be needed for contents such as washer, dryer, TV, or food freezer. <br/>'+
                '(4) Consider funding for mental health support of household members who experienced flooding. <br/>'+
                '(5) Fostering relationships with residents and community organizations is vital to establish trust and make buyouts more favorable. <br/>'+
                '(6) Consider conducting longitudinal health studies to assess and address potential physical and mental health implications for participants undergoing the relocation processes in buyout projects. <br/>'+
                '(7) Partnering with community-based organizations and assigning local relocation specialists helps to ensuring an equitable and supportive buyout process, given their role in navigating and addressing the complexities of relocation. <br/>'+
                '(8) To counter tax base losses from buyouts, program administrators can incentivize owners to relocate within the county but outside floodplains. <br/>'+
                '(9) If a participating household owes a mortgage, FEMA may pay residual funds after they pay the lienholder. <br/>'+
                '(10) Effective risk communication does not always lead to protective actions like participating in a buyout program. Homeowners responses depend on assessing benefits and their ability to act. Thus, sharing information on buyout benefits, resources like temporary housing options, and available compensations increases confidence in homeowners ability to afford and undergo buyouts.'],    
    'Renters': [renters, f"${normal_offer_renters:,.2f}", benefit_renter,
                '(1) Renters-occupied homes are less likely to undergo buyouts. <br/>'+
                '(2) Foreign national renters are ineligible for federal relocation benefits and post-buyout assistance, intensifying challenges for local governments in providing alternative solutions.',
                '(1) Mandatory relocation <br/>'+
                '(2) Financial burden <br/>'+
                '(3) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability <br/>'+
                '(4) Losing social networks if relocated outside of the community <br/>'+
                '(5) Lower ability to find affordable homes when relocated',  
                '(1) Renters are less likely to carry flood insurance. Local governments may explore strategies to enhance coverage and overall disaster preparedness. <br/>'+
                '(2) Under the URA of 1970, displaced tenants from a property (occupied for 90+ days) can receive a payment (up to $5,250) for renting another property for up to 42 months. <br/>'+
                '(3) Consider funding for mental health support of household members who experienced flooding. <br/>'+
                '(4) Consider conducting longitudinal health studies to assess and address potential physical and mental health implications for participants undergoing the relocation processes in buyout projects. <br/>'+
                '(5) Partnering with community-based organizations and assigning local relocation specialists helps to ensuring an equitable and supportive buyout process, given their role in navigating and addressing the complexities of relocation.']
}


ownership_table = pd.DataFrame(ownership_summary, index=None)


ownership_table = ownership_table.style.set_properties(
    subset=ownership_table.columns[1:],  
    **{'text-align': 'left'}
).set_properties(
    subset=ownership_table.columns[0], 
    **{'font-weight': 'bold', 'text-align': 'left'}
).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]}
])
ownership_table

,,Owners,Renters
0,# candidate housing units,8,12
1,Total purchase offer,"$1,372,004.00","$1,203,804.00"
2,Potential benefits (# people not dislocated by a simulated 100-year event),5,10
3,Potential challenges,"(1) Program participation depends on financial standing, place-based attachments, risk perception, flood exposure, family composition, and community ties. (2) Homeowners are less likely to participate if they have a mortgage. (3) Elderly homeowners (with paid off mortgages) may oppose relocation, due to unwillingness to take out a second mortgage for a more expensive home and place-based attachments. (4) Homeowners tend to decline offers if their property has undergone improvements, as home improvements are not factored into the market value. (5) Lower chance of accepting an offer if cost sharing is required. (6) Lower chance of accepting an offer if their house has undergone repair and reconstruction.(7) Owners of substantially damaged properties may have limited post-disaster rebuilding options. While eminent domain is prohibited in buyout programs, the owners are legally restricted from rebuilding unless they flood-proof the structure or relocate, increasing their participation likelihood. (8) Homeowners are less inclined to participate if the purchase offer is lower than their expected property value. (9) Homeowners with inadequate property documentation are less likely to participate.","(1) Renters-occupied homes are less likely to undergo buyouts. (2) Foreign national renters are ineligible for federal relocation benefits and post-buyout assistance, intensifying challenges for local governments in providing alternative solutions."
4,Potential consequences,(1) Possibility of unaffordable housing options upon relocation (2) Losing social network if relocated outside their community,(1) Mandatory relocation (2) Financial burden (3) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability (4) Losing social networks if relocated outside of the community (5) Lower ability to find affordable homes when relocated
5,Equity considerations,"(1) Sufficient financial incentives can alleviate post-buyout financial burdens and encourage program participation. (2) Up to $22,500 compensation under URA for additional payments such as comparable homes, closing costs, or increased interest costs. (3) Basement coverage through FEMA’s NFIP is limited, and compensation may be needed for contents such as washer, dryer, TV, or food freezer. (4) Consider funding for mental health support of household members who experienced flooding. (5) Fostering relationships with residents and community organizations is vital to establish trust and make buyouts more favorable. (6) Consider conducting longitudinal health studies to assess and address potential physical and mental health implications for participants undergoing the relocation processes in buyout projects. (7) Partnering with community-based organizations and assigning local relocation specialists helps to ensuring an equitable and supportive buyout process, given their role in navigating and addressing the complexities of relocation. (8) To counter tax base losses from buyouts, program administrators can incentivize owners to relocate within the county but outside floodplains. (9) If a participating household owes a mortgage, FEMA may pay residual funds after they pay the lienholder. (10) Effective risk communication does not always lead to protective actions like participating in a buyout program. Homeowners responses depend on assessing benefits and their ability to act. Thus, sharing information on buyout benefits, resources like temporary housing options, and available compensations increases confidence in homeowners ability to afford and undergo buyouts.","(1) Renters are less likely to carry flood insurance. Local governments may explore strategies to enhance coverage and overall disaster preparedness. (2) Under the URA of 1970, displ

In [66]:
df_final.crs = "EPSG:4326"
df_final = df_final.to_crs(epsg=3857)

In [67]:
color_dict = {
    'Owner': 'blue',
    'Renter': 'red'
}

# Create a ListedColormap with your specific colors
colors = [color_dict[ownership] for ownership in df_final['ownership'].unique()]
cmap = ListedColormap(colors)

df_final.explore(
        column="ownership", # column for archetype info
        tooltip="ownership", 
        geometry='geometry',
        popup=True,
        tiles='OpenStreetMap',
        cmap=cmap,  # use "Set1" matplotlib colormap
        legend_kwds = {"caption": "Ownership"},
        marker_kwds= {'radius': 4}
)

### 2) Owner-occupied homes broken down by owners' race

In [68]:
white = df_final.loc[(df_final['race'] == 1) & (df_final['hispan'] == 0) & (df_final['ownership'] == 'Owner'), 'huid'].count()
racial_minority = df_final.loc[(df_final['race'] >= 2) & (df_final['hispan'] == 0) & (df_final['ownership'] == 'Owner'), 'huid'].count()
hisp = df_final.loc[(df_final['hispan'] == 1) & (df_final['ownership'] == 'Owner'), 'huid'].count()

offer_owner_white = round(df_final.loc[(df_final['race'] == 1) & (df_final['hispan'] == 0) & (df_final['ownership'] == 'Owner'), 'housing_unit_appraisal_value'].sum())
offer_owner_racial_minority = round(df_final.loc[(df_final['race'] >= 2) & (df_final['hispan'] == 0) & (df_final['ownership'] == 'Owner'), 'housing_unit_appraisal_value'].sum())
offer_owner_hispan = round(df_final.loc[(df_final['hispan'] == 1) & (df_final['ownership'] == 'Owner'), 'housing_unit_appraisal_value'].sum())

benefit_owner_white = df_final.loc[(df_final['dislocated'] == True) & (df_final['ownership'] == 'Owner') & (df_final['race'] == 1) & (df_final['hispan'] == 0), 'dislocated'].sum()
benefit_owner_racial_minority = df_final.loc[(df_final['dislocated'] == True) & (df_final['ownership'] == 'Owner') & (df_final['race'] >= 2) & (df_final['hispan'] == 0), 'dislocated'].sum()
benefit_owner_hispan = df_final.loc[(df_final['dislocated'] == True) & (df_final['ownership'] == 'Owner') & (df_final['hispan'] == 1), 'dislocated'].sum()

# NOTE: Equity considerations are similar for all homeowners
tenure_race_summary = {
    '': ['# candidate housing units', 'Total purchase offer', 'Potential benefits (# people not dislocated by a simulated 100-year event)',
         'Potential challenges','Potential consequences', 'Equity considerations'],
    'White homeowners': [white, f"${offer_owner_white:,.2f}", benefit_owner_white,
                '(1) Less likely to participate due to higher flood risk tolerance.',
                '(1) If agree to participate, may prefer relocating to majority-White communities.', 
                '(1) Sufficient financial incentives can alleviate post-buyout financial burdens and encourage program participation. <br/>'+
                '(2) Up to $22,500 compensation under URA for additional payments such as comparable homes, closing costs, or increased interest costs. <br/>'+
                '(3) Basement coverage through FEMA’s NFIP is limited, and compensation may be needed for contents such as washer, dryer, TV, or food freezer. <br/>'+
                '(4) Consider funding for mental health support of household members who experienced flooding. <br/>'+
                '(5) Fostering relationships with residents and community organizations is vital to establish trust and make buyouts more favorable. <br/>'+
                '(6) Consider conducting longitudinal health studies to assess and address potential physical and mental health implications for participants undergoing the relocation processes in buyout projects. <br/>'+
                '(7) Partnering with community-based organizations and assigning local relocation specialists helps to ensuring an equitable and supportive buyout process, given their role in navigating and addressing the complexities of relocation. <br/>'+
                '(8) To counter tax base losses from buyouts, program administrators can incentivize owners to relocate within the county but outside floodplains. <br/>'+
                '(9) If a participating household owes a mortgage, FEMA may pay residual funds after they pay the lienholder. <br/>'+
                '(10) Effective risk communication does not always lead to protective actions like participating in a buyout program. Homeowners responses depend on assessing benefits and their ability to act. Thus, sharing information on buyout benefits, resources like temporary housing options, and available compensations increases confidence in homeowners ability to afford and undergo buyouts.'],    
    'Racial minority homeowners': [racial_minority, f"${offer_owner_racial_minority:,.2f}", benefit_owner_racial_minority,
                '(1) More prone to relocation due to higher exposure to disasters <br/>'+
                '(2) Reluctance to join the program due to emotional attachment and distrust in government-led programs <br/>'+
                '(3) Properties in Black neighborhoods are valued lower compared to White neighborhoods, posing challenges for finding comparable homes post-buyout.',
                '(1) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability. <br/>'+
                '(2) Losing social network if relocated outside their community',
                ' '],
    'Hispanic homeowners': [hisp, f"${offer_owner_hispan:,.2f}", benefit_owner_hispan,
                '(1) More prone to relocation due to higher exposure to disasters <br/>'+
                '(2) Properties in Hispanic neighborhoods are valued lower compared to White neighborhoods, posing challenges for finding comparable homes post-buyout <br/>'+
                '(3) Reluctance to join the program due to emotional attachment and distrust in government-led programs',
                '(1) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability <br/>'+
                '(2) Losing social network if relocated outside their community',
                ' '],
}

tenure_race_table = pd.DataFrame(tenure_race_summary, index=None)

tenure_race_table = tenure_race_table.style.set_properties(
    subset=tenure_race_table.columns[1:],  
    **{'text-align': 'left'}
).set_properties(
    subset=tenure_race_table.columns[0],  
    **{'font-weight': 'bold', 'text-align': 'left'}
).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]}
])

tenure_race_table

,,White homeowners,Racial minority homeowners,Hispanic homeowners
0,# candidate housing units,7,0,1
1,Total purchase offer,"$1,082,784.00",$0.00,"$289,220.00"
2,Potential benefits (# people not dislocated by a simulated 100-year event),4,0,1
3,Potential challenges,(1) Less likely to participate due to higher flood risk tolerance.,"(1) More prone to relocation due to higher exposure to disasters (2) Reluctance to join the program due to emotional attachment and distrust in government-led programs (3) Properties in Black neighborhoods are valued lower compared to White neighborhoods, posing challenges for finding comparable homes post-buyout.","(1) More prone to relocation due to higher exposure to disasters (2) Properties in Hispanic neighborhoods are valued lower compared to White neighborhoods, posing challenges for finding comparable homes post-buyout (3) Reluctance to join the program due to emotional attachment and distrust in government-led programs"
4,Potential consequences,"(1) If agree to participate, may prefer relocating to majority-White communities.",(1) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability. (2) Losing social network if relocated outside their community,(1) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability (2) Losing social network if relocated outside their community
5,Equity considerations,"(1) Sufficient financial incentives can alleviate post-buyout financial burdens and encourage program participation. (2) Up to $22,500 compensation under URA for additional payments such as comparable homes, closing costs, or increased interest costs. (3) Basement coverage through FEMA’s NFIP is limited, and compensation may be needed for contents such as washer, dryer, TV, or food freezer. (4) Consider funding for mental health support of household members who experienced flooding. (5) Fostering relationships with residents and community organizations is vital to establish trust and make buyouts more favorable. (6) Consider conducting longitudinal health studies to assess and address potential physical and mental health implications for participants undergoing the relocation processes in buyout projects. (7) Partnering with community-based organizations and assigning local relocation specialists helps to ensuring an equitable and supportive buyout process, given their role in navigating and addressing the complexities of relocation. (8) To counter tax base losses from buyouts, program administrators can incentivize owners to relocate within the county but outside floodplains. (9) If a participating household owes a mortgage, FEMA may pay residual funds after they pay the lienholder. (10) Effective risk communication does not always lead to protective actions like participating in a buyout program. Homeowners responses depend on assessing benefits and their ability to act. Thus, sharing information on buyout benefits, resources like temporary housing options, and available compensations increases confidence in homeowners ability to afford and undergo buyouts.",,


In [69]:
def categorize_race_tenure(row):
    if row['race'] == 1 and row['hispan'] == 0 and row['ownership'] == 'Owner':
        return 'White alone homeowners'
    elif row['hispan'] == 1 and row['ownership'] == 'Owner':
        return 'Hispanic homeowners'
    elif row['race'] >= 2 and row['hispan'] == 0 and row['ownership'] == 'Owner':
        return 'Racial minority homeowners'
    else:
        return 'Other'


# Apply the function to each row
df_final['race_category'] = df_final.apply(categorize_race_tenure, axis=1)

colors = ['red','green','blue']
cmap = ListedColormap(colors)

df_final[df_final['race_category'] != 'Other'].explore(
        column="race_category", # column for archetype info
        tooltip="race_category", 
        geometry='geometry',
        popup=True,
        tiles='OpenStreetMap',
        cmap=cmap,  # use "Set1" matplotlib colormap
        legend_kwds = {"caption": "Race Category"},
        marker_kwds= {'radius': 4}
)

### 3) Owner-occupied homes broken down by owners' income level

In [70]:
low_middle_income = df_final.loc[(df_final['hhinc'].isin([1,2,3])) & (df_final['ownership'] == 'Owner'), 'huid'].count()
high_income = df_final.loc[(df_final['hhinc'].isin([4,5])) & (df_final['ownership'] == 'Owner'), 'huid'].count()

offer_owner_low_middle_income = round(df_final.loc[(df_final['hhinc'].isin([1,2,3])) & (df_final['ownership'] == 'Owner'), 'housing_unit_appraisal_value'].sum())
offer_owner_high_income = round(df_final.loc[(df_final['hhinc'].isin([4,5])) & (df_final['ownership'] == 'Owner'), 'housing_unit_appraisal_value'].sum())

benefit_owner_low_middle_income = df_final.loc[(df_final['hhinc'].isin([1,2,3])) & (df_final['ownership'] == 'Owner'), 'dislocated'].sum()
benefit_owner_high_income = df_final.loc[(df_final['hhinc'].isin([4,5])) & (df_final['ownership'] == 'Owner'), 'dislocated'].sum()

# NOTE: Equity considerations are similar for all homeowners
income_tenure_summary = {
    '': ['# candidate housing units', 'Total purchase offer', 'Potential benefits (# people not dislocated by a simulated 100-year event)',
         'Potential challenges','Potential consequences', 'Equity considerations'],
    'Low and middle income homeowners': [low_middle_income, f"${offer_owner_low_middle_income:,.2f}", benefit_owner_low_middle_income,
                '(1) Higher chance of being targeted for program participation as low-value homes sustain more damage <br/>'+
                '(2) Lower chance of accepting an offer if cost sharing is required <br/>'+
                '(3) May accept a low offer due to little power to negotiate the terms <br/>'+
                '(4) Reluctance to relocate due to limited perceived options, lower awareness, and affordability concerns <br/>' +
                '(5) More prone to relocation due to higher exposure to disasters',
                '(1) Less likely to find affordable comparable homes when relocated <br/>'+
                '(2) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability <br/>'+
                '(3) Losing social networks if relocated outside of the community <br/>'+
                '(4) Though voluntary, program participation may resemble forced relocation due to new post-disaster policies mandating costly mitigation measures or increased insurance premiums that are unaffordable to households.',
                '(1) Sufficient financial incentives can alleviate post-buyout financial burdens and encourage program participation. <br/>'+
                '(2) Up to $22,500 compensation under URA for additional payments such as comparable homes, closing costs, or increased interest costs. <br/>'+
                '(3) Basement coverage through FEMA’s NFIP is limited, and compensation may be needed for contents such as washer, dryer, TV, or food freezer. <br/>'+
                '(4) Consider funding for mental health support of household members who experienced flooding. <br/>'+
                '(5) Fostering relationships with residents and community organizations is vital to establish trust and make buyouts more favorable. <br/>'+
                '(6) Consider conducting longitudinal health studies to assess and address potential physical and mental health implications for participants undergoing the relocation processes in buyout projects. <br/>'+
                '(7) Partnering with community-based organizations and assigning local relocation specialists helps to ensuring an equitable and supportive buyout process, given their role in navigating and addressing the complexities of relocation. <br/>'+
                '(8) To counter tax base losses from buyouts, program administrators can incentivize owners to relocate within the county but outside floodplains. <br/>'+
                '(9) If a participating household owes a mortgage, FEMA may pay residual funds after they pay the lienholder. <br/>'+
                '(10) Effective risk communication does not always lead to protective actions like participating in a buyout program. Homeowners responses depend on assessing benefits and their ability to act. Thus, sharing information on buyout benefits, resources like temporary housing options, and available compensations increases confidence in homeowners ability to afford and undergo buyouts.'],    
    'High income homeowners': [high_income, f"${offer_owner_high_income:,.2f}", benefit_owner_high_income,
                '(1) Higher chance of ineligibility due to owning high-value properties and not meeting the BCA requirements <br/>'+
                '(2) May not receive a buyout offer as municipalities are concerned about the appearance of federal funds assisting wealthy households <br/>'+
                '(3) May decline the offer due to more financial independence and self-efficacy to implement alternative mitigation measures <br/>'+
                '(4) May decline offers if their property has undergone improvements, as home improvements are not factored into the market value',
                '(1) May need to adopt alternative mitigation measures, if not participating',
                ' ']
}

income_tenure_table = pd.DataFrame(income_tenure_summary, index=None)

income_tenure_table = income_tenure_table.style.set_properties(
    subset=income_tenure_table.columns[1:],  
    **{'text-align': 'left'}
).set_properties(
    subset=income_tenure_table.columns[0],  
    **{'font-weight': 'bold', 'text-align': 'left'}
).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]}
])

income_tenure_table

,,Low and middle income homeowners,High income homeowners
0,# candidate housing units,3,5
1,Total purchase offer,"$507,766.00","$864,237.00"
2,Potential benefits (# people not dislocated by a simulated 100-year event),2,3
3,Potential challenges,"(1) Higher chance of being targeted for program participation as low-value homes sustain more damage (2) Lower chance of accepting an offer if cost sharing is required (3) May accept a low offer due to little power to negotiate the terms (4) Reluctance to relocate due to limited perceived options, lower awareness, and affordability concerns (5) More prone to relocation due to higher exposure to disasters","(1) Higher chance of ineligibility due to owning high-value properties and not meeting the BCA requirements (2) May not receive a buyout offer as municipalities are concerned about the appearance of federal funds assisting wealthy households (3) May decline the offer due to more financial independence and self-efficacy to implement alternative mitigation measures (4) May decline offers if their property has undergone improvements, as home improvements are not factored into the market value"
4,Potential consequences,"(1) Less likely to find affordable comparable homes when relocated (2) Higher chance of relocating to areas with equal or greater flood exposure and higher social vulnerability (3) Losing social networks if relocated outside of the community (4) Though voluntary, program participation may resemble forced relocation due to new post-disaster policies mandating costly mitigation measures or increased insurance premiums that are unaffordable to households.","(1) May need to adopt alternative mitigation measures, if not participating"
5,Equity considerations,"(1) Sufficient financial incentives can alleviate post-buyout financial burdens and encourage program participation. (2) Up to $22,500 compensation under URA for additional payments such as comparable homes, closing costs, or increased interest costs. (3) Basement coverage through FEMA’s NFIP is limited, and compensation may be needed for contents such as washer, dryer, TV, or food freezer. (4) Consider funding for mental health support of household members who experienced flooding. (5) Fostering relationships with residents and community organizations is vital to establish trust and make buyouts more favorable. (6) Consider conducting longitudinal health studies to assess and address potential physical and mental health implications for participants undergoing the relocation processes in buyout projects. (7) Partnering with community-based organizations and assigning local relocation specialists helps to ensuring an equitable and supportive buyout process, given their role in navigating and addressing the complexities of relocation. (8) To counter tax base losses from buyouts, program administrators can incentivize owners to relocate within the county but outside floodplains. (9) If a participating household owes a mortgage, FEMA may pay residual funds after they pay the lienholder. (10) Effective risk communication does not always lead to protective actions like participating in a buyout program. Homeowners responses depend on assessing benefits and their ability to act. Thus, sharing information on buyout benefits, resources like temporary housing options, and available compensations increases confidence in homeowners ability to afford and undergo buyouts.",


In [71]:
def categorize_income(row):
    if row['hhinc'] in [1, 2, 3] and row['ownership'] == 'Owner':
        return 'Low and middle income homeowners'
    elif row['hhinc'] in [4, 5] and row['ownership'] == 'Owner':
        return 'High income homeowners'
    else:
        return 'Other'

# Apply the function to each row
df_final['income_category'] = df_final.apply(categorize_income, axis=1)
colors = ['red','blue']
cmap = ListedColormap(colors)

df_final[df_final['income_category'] != 'Other'].explore(
        column="income_category", # column for archetype info
        tooltip="income_category", 
        geometry='geometry',
        popup=True,
        tiles='OpenStreetMap',
        cmap=cmap,
        legend_kwds = {"caption": "Income Group"},
        marker_kwds= {'radius': 4}
)

### 4) Cluster buyout

In [ ]:
# Calculating the distance of closest building to each building in the inventory
# geometry_col = gpd.GeoSeries.from_wkt(bld_invtry['geometry'])
distance = gpd.GeoDataFrame(geometry=bld_invtry['geometry'])

# Setting the CRS to WGS 84
distance = distance.set_crs(epsg=4326)

# Converting the CRS to a projection that uses feet (e.g., UTM)
distance = distance.to_crs(epsg=3857) 
bld_invtry['closest_distance'] = distance.geometry.apply(lambda x: distance.distance(x).nsmallest(2).iloc[1])

In [ ]:
bld_invtry['closest_distance'].describe()

In [ ]:
df_cluster=df_final

In [ ]:
# Setting the threshold for identifying clusters, which is 10 times the standard deviation of calcualted distance
cluster_threshold = 10 * np.std(bld_invtry['closest_distance'])

def identify_clusters(coord1, coord2):
    return geodesic(coord1, coord2).feet
df_cluster['cluster'] = ''

group_counter = 1

for i in range(len(df_cluster)):
    if df_cluster.at[i, 'cluster'] == '':
        df_cluster.at[i, 'cluster'] = f'Group_{group_counter}'
        group_counter += 1

        for j in range(i + 1, len(df_cluster)):
            coord1 = (df_cluster.at[i, 'y'], df_cluster.at[i, 'x'])
            coord2 = (df_cluster.at[j, 'y'], df_cluster.at[j, 'x'])
            distance = identify_clusters(coord1, coord2)

            if distance <= cluster_threshold:
                df_cluster.at[j, 'cluster'] = df_cluster.at[i, 'cluster']

pd.set_option('display.float_format', '{:.15f}'.format)

In [ ]:
# keeping only one housing unit per building to avoid getting clusters just because one building has two housing units (drop duplicate)
df_cluster = df_cluster.drop_duplicates(subset='guid', keep='first')

In [ ]:
# Removing properties that are not in a cluster by dropping the values in "cluster" col that are repeated only once
df_cluster = df_cluster[df_cluster.groupby('cluster')['cluster'].transform('count') > 1]
#df_cluster.shape

In [ ]:
unique_clusters = df_cluster['cluster'].unique()
mapping_dict = {group: f'Cluster {i + 0}' for i, group in enumerate(unique_clusters)}

df_cluster['cluster_label'] = df_cluster['cluster'].map(mapping_dict)
df_cluster = gpd.GeoDataFrame(df_cluster, geometry='geometry')

df_cluster.explore(
        column="cluster_label", # column for archetype info
        tooltip="cluster_label", 
        geometry='geometry',
        popup=True,
        tiles='OpenStreetMap',
        cmap="tab20b",  # use "Set1" matplotlib colormap
        legend_kwds = {"caption": "Clusters"},
        marker_kwds= {'radius': 4}
)

In [ ]:
cluster_buyout_summary = {
    'Potential advantages of cluster buyout': [
        '(1) Incentivizes participation if other neighbors are relocating <br/>' +
        '(2) Losing tax base is a known issue of buyout. Recreational areas developed through cluster buyout, such as parks help increase property values and improve physical and mental health <br/>' +
        '(3) Boosts tourism and recreation by creating new spaces <br/>' +
        '(4) Helps restoring ecological values and enhancing ecosystem services such as improved water quality and reduced risks of future hazards <br/>' +
        '(5) Cluster buyouts align with FEMA open space management objectives of FEMA. While certain communities have repurposed checkerboard-patterned lots for recreation, these lots generally limit open space development opportunities.'
    ]
}

cluster_buyout_table = pd.DataFrame(cluster_buyout_summary, index=None)

cluster_buyout_table = cluster_buyout_table.style.set_properties(
    subset=cluster_buyout_table.columns,  
    **{'text-align': 'left'}
).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]}
])

cluster_buyout_table

### 5) Properties (valued higher than FEMA's cap) requiring alternative mitigation strategies

In [ ]:
ike_100year_ds3_over_fema_cap = ike_100year_ds3[ike_100year_ds3['appr_bldg'] > 321291.600]
ike_100year_ds3_over_fema_cap = ike_100year_ds3_over_fema_cap[["guid","appr_bldg","max_state_100year","max_state_ike","geometry"]]

In [ ]:
# Converting the dataset to GeoDataFrame using the goemetry column 
ike_100year_ds3_over_fema_cap['geometry'] = ike_100year_ds3_over_fema_cap['geometry']
ike_100year_ds3_over_fema_cap = gpd.GeoDataFrame(ike_100year_ds3_over_fema_cap, geometry='geometry')

In [ ]:
ike_100year_ds3_over_fema_cap['guid'].count()

In [ ]:
ike_100year_ds3_over_fema_cap.crs = "EPSG:4326"
ike_100year_ds3_over_fema_cap = ike_100year_ds3_over_fema_cap.to_crs(epsg=3857)

In [ ]:
ike_100year_ds3_over_fema_cap.explore(
    geometry='geometry',
    popup=True,
    tiles='OpenStreetMap',
    marker_kwds={'radius': 4},
    title="Properties with past and future complete damages valued higher than FEMA's cap",  # Add title here
)

In [ ]:
property_over_cap = ike_100year_ds3_over_fema_cap['guid'].count()
description = (
    f"There are {property_over_cap} properties with complete damages as a result of Ike's hindcast model "
    "and the simulated 100-year event.\nThese properties are not in the buyout selection due to high property "
    "value, and require alternative flood mitigation strategies."
)
print(description)